In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import requests 
import io
import seaborn as sns

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize

import string
from nltk import word_tokenize
from nltk.corpus import stopwords

from bs4 import BeautifulSoup

nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from transformers import BertTokenizer,BertModel, BertConfig
import torch

### User Mapping

In [ ]:
orig_members = pd.read_csv('kad_mrgcn_public/data/ironmarch/raw_files/orig_members.csv')

In [ ]:
sorted_member_id = orig_members['member_id'].sort_values()
member_id_map ={}
start_id = 1
for id in sorted_member_id:
  member_id_map[id] = start_id
  start_id = start_id + 1

### Message Posts

In [ ]:
orig_posts = pd.read_csv('kad_mrgcn_public/data/ironmarch/raw_files/orig_posts.csv')

In [ ]:
orig_posts.drop(orig_posts.columns[[0,1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]], axis =1, inplace=True)

### Functions

In [ ]:
def preprocess_text(text):

    soup = BeautifulSoup(text, "lxml")
    text = soup.body.text

    # Get a list of sentences, removes "\n"
    lst = sent_tokenize(text)
    text = " ".join(lst)

    text = text.replace("\n", "")

    # Preprocess using NLTK
    text = text.lower()

    # Remove Punctuation
    text_p = "".join([char for char in text if char not in string.punctuation])

    # Get a List of words 
    words = word_tokenize(text_p)

    words = " ".join(words)

    # Remove stopwords
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]

    ret = " ".join(filtered_words)

    return ret


In [ ]:
def getBERT(post):

  tokens = tokenizer.encode_plus(post,return_tensors='pt', 
                                 max_length=512, truncation=True)
  outputs = model(**tokens)
  last_hidden_states = outputs.last_hidden_state

# Take the average
  n_tensor = last_hidden_states.shape[1]
  vector = (last_hidden_states.sum(axis=1)/n_tensor)

  return vector

### Variables

In [ ]:
embedding = np.zeros((orig_members.shape[0], 768))
count = np.zeros(orig_members.shape[0])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Iterations

In [ ]:
for i, row in orig_posts.iterrows():
  author_id = row['author_id']
  if author_id not in member_id_map:
          continue
  id = member_id_map[author_id]
  doc = row['post']

  clean_doc = preprocess_text(doc)

  
  embed_doc = getBERT(clean_doc)
  embed_doc = embed_doc.detach().numpy()


  embedding[id-1] = (count[id-1]*embedding[id-1] + embed_doc)/(count[id-1]+1)
  count[id-1] = count[id-1]+1

  if i%5000==0:
    num = int(i/5000)
    np.save("kad_mrgcn_public/data/ironmarch/processed/posts_embedding_head_v{}".format(num+1), embedding)


In [ ]:
np.save("kad_mrgcn_public/data/ironmarch/processed/posts_embedding_head_final", embedding_final)